In [ ]:
public function updateDestinationTarget()
{
    // Si la valeur est vide, on met à null pour indiquer une livraison client directe
    $target = $this->depot_destination_id ?: null;

    $this->produit->update([
        'depot_destination_id' => $target
    ]);
    
    $this->produit->refresh();
    
    $msg = $target ? 'Cible définie : Dépôt' : 'Cible définie : Client Final';
    session()->flash('message', $msg);
}

In [ ]:
public function getCanManageProperty()
{
    $user = auth()->user();

    if ($user->role === 'super_admin') {
        return true;
    }

    if ($user->role !== 'admin') {
        return false;
    }

    // Récupérer le dépôt géré par cet admin
    $adminDepot = Depot::where('admin_id', $user->id)->first();

    if (!$adminDepot) {
        return false; // Un admin sans dépôt ne gère rien (ou fallback ?)
    }

    // Le produit est gérable SI :
    // 1. Il est actuellement dans ce dépôt
    if ($this->produit->depot_actuel_id === $adminDepot->id) {
        return true;
    }

    // 2. Cas spécial : Le produit vient d'être créé (pas de dépôt actuel) et c'est le dépôt source ?
    // Ou si admin est le créateur ? 
    // Pour l'instant, on se base strictement sur la localisation actuelle.
    
    return false;
}